In [1]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00


In [2]:
import os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, EarlyStoppingCallback
import torch

# Load the model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundation_model = AutoModelForCausalLM.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:

# Load and preprocess the dataset
data = load_dataset("cnn_dailymail", "3.0.0")
tokenizer.pad_token = tokenizer.eos_token

# Using only a small sample of the data
train_size, test_size, validation_size = [int(0.01 * len(data[split])) for split in ['train', 'test', 'validation']]
train_sample = data['train'].select(range(train_size))
test_sample = data['test'].select(range(test_size))
validation_sample = data['validation'].select(range(validation_size))

# Preprocessing function
def preprocess_function(examples):
    model_inputs = tokenizer(examples["article"], max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_data_train = train_sample.map(preprocess_function, batched=True)
tokenized_data_test = test_sample.map(preprocess_function, batched=True)
tokenized_data_validation = validation_sample.map(preprocess_function, batched=True)


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Map:   0%|          | 0/2871 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/133 [00:00<?, ? examples/s]

In [4]:
import transformers

In [9]:
# Freeze all layers initially
for param in foundation_model.parameters():
    param.requires_grad = False

# Unfreeze the last Transformer block
for param in foundation_model.transformer.h[-1].parameters():
    param.requires_grad = True

# Also unfreeze `lm_head` if you'd like it to adjust to token outputs directly
for param in foundation_model.lm_head.parameters():
    param.requires_grad = True

# Check the number of parameters to verify
total_params = sum(p.numel() for p in foundation_model.parameters())
trainable_params = sum(p.numel() for p in foundation_model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 124439808
Trainable parameters: 45685248


In [11]:

# Define the output directory for the fine-tuned model
output_directory = "./gpt2_finetuned_last_layers"

# Define training arguments
training_args = TrainingArguments(
    output_dir=output_directory,
    evaluation_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    save_steps=50,
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    report_to="none",
    no_cuda=False,
    load_best_model_at_end=True,
    metric_for_best_model='loss',
    greater_is_better=False,
)

# Initialize Trainer
trainer = Trainer(
    model=foundation_model,
    args=training_args,
    train_dataset=tokenized_data_train,
    eval_dataset=tokenized_data_validation,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Start training
trainer.train()


Step,Training Loss,Validation Loss
50,3.228700,3.116729
100,3.171400,3.130275
150,3.148800,3.127252
200,3.113500,3.131474


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=200, training_loss=3.1656334686279295, metrics={'train_runtime': 139.3501, 'train_samples_per_second': 61.808, 'train_steps_per_second': 15.457, 'total_flos': 209033625600000.0, 'train_loss': 3.1656334686279295, 'epoch': 0.2785515320334262})

In [14]:

# Evaluate the model
test_results = trainer.evaluate(tokenized_data_test)
print("Test results:", test_results)

Test results: {'eval_loss': 3.0684139728546143, 'eval_runtime': 4.871, 'eval_samples_per_second': 23.404, 'eval_steps_per_second': 3.079, 'epoch': 0.2785515320334262}
